# How to use DenseSSD? (Education version)

With the aim of improving safety in a surveillance-free laboratory, we report a novel deep learning (DL)-based object detector, namely, `DenseSSD`.

So, in this notebook, we share how to use our `DenseSSD` and explain more detailed about code.

## Module import

please import as you use in this ipynb notebook. If there is a module that you want to use more, you can import it.

- torch (1.7.0)
- numpy
- os
- PIL
- natsort
- warning

In [ ]:
from PIL import Image
from natsort import natsorted
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
import os
import numpy as np
import torch

## Our module

### 1. config.py

config.py include information of executing our code, such as GPU device, path, hyperparameter.

[GPU Device]
- device = "cuda:0"

[Path to data dir]
- test_sample_image_dir = 'dataset/test_sample'
- test_result_image_dir = 'dataset/test_result'
- image_dir_train = 'dataset/training_sample/Empty/img'
- info_dir_train  = 'dataset/training_sample/Empty/annotation'
- train_label     = 'train.txt'
- val_label       = 'valid.txt'
- model_path = 'pre-trained/model.pth' : set pre-trained model path if we have pre-trained model

[Hyper parameters]
- image_size = 300 : set our image size
- C : input the number of candidates, (ex. C=3, generate 3 candidate boxes of each objects)
- batch_size : input batch_size, it depends on researchers
- init_lr : input initial learning rate, it depends on num_epochs (if you control steps of lr reduction, please looking for update_lr function in train.py )
- weight_decay : input weight_decay, it depends on researchers
- num_epochs : input epochs, it depends on researchers

### 2. model.py --> denseSSD
class denseSSD  
├── denseNet  (denseblock)  
│   └── Bottleneck  
│   └── Transition  
│   └── Dimension_Reduction  
└── PredictionLayer  

`model.py` include the architecture of `DenseSSD` model using pytorch.
And those architecture follow our [architecture's image](info/model_architecture.png)

In [ ]:
# import our module
import config # config
from model.denseSSD import denseSSD # our detection model
from train import train # train function
from detect import visualize_detection # test function

## Model upload
After we import all of module that we use, we have 2 types of process that we select.
1. Train new model
2. Load pre-trained model

So, please check your process type and select just only one type.

### 1. Train new model

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# if you don't have any pre-trained model?
print('---------------------------------')
print("Train new model!")
print('---------------------------------')
model = denseSSD(n_classes=config.C)
device = torch.device(config.device) # match GPU
model.to(device) # allocate our model on GPU
train(model, config)

### 2. Load pre-trained model

In [ ]:
# if you have pretrained model?
print('---------------------------------')
print("Loading pre-trained model!")
print('---------------------------------')
model = denseSSD(n_classes=config.C)
model_path = config.model_path
model.load_state_dict(torch.load(model_path)) 
model.to(config.device) # allocate our model on GPU
model.eval()

## Test a few samples

In [ ]:
print('---------------------------------')
print("Load testing samples!")
print('---------------------------------')
image_dir = config.test_sample_image_dir
files = os.listdir(image_dir)
files = natsorted(files)

In [ ]:
print('---------------------------------')
print("Detection begins!")
print('---------------------------------')
for file in files:
    image_path = os.path.join(image_dir, file)
    original_image = Image.open(image_path, mode='r')
    original_image = original_image.convert('RGB')

    _, objects = visualize_detection(model, original_image, min_score=0.2, max_overlap=0.5, top_k=200,
                                        path=config.test_result_image_dir+"/"+file)
    print(config.test_result_image_dir+"/"+file)
    print("Test scene file - %s: %d vials detected!" % (file, objects))
print("\nTest completed!\n")